# UNIDAD 4: Uso de LLMs mediante APIs - Parte I (Gemini)



The **Gemini model** family includes models that work with multimodal prompt requests. The term multimodal indicates that you can use more than one modality, or type of input, in a prompt. Models that aren't multimodal accept prompts only with text. Modalities can include text, audio, video, and more.

- Documentación de la API: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#python_1

To **request access**, go ton: https://aistudio.google.com/


In [3]:
import requests
import json
from dotenv import load_dotenv
import os

# Warning control
import warnings
warnings.filterwarnings("ignore")

<b>Obtención de la API key desde archivo de variables de entorno y definición del Endpoint de Gemini</b>

In [4]:
assert load_dotenv('env', override=True)

# API de Gemini:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_API_KEY

# URL de la API de Gemini
url = f"https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={GOOGLE_API_KEY}"

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

opinions = pd.read_csv('../commentsEC.csv', names = ['id', 'text', 'rating'])
opinions.head(3)

,id,text,rating
0,1,"""Exploring the historic city of #Quito was amazing! Can't wait to come back! 🇪🇨 @VisitEcuador https://t.co/quito_adventures""",5
1,2,"""The Galápagos Islands are truly a paradise! 🐢😍 #Galapagos #WildlifeTravel https://t.co/galapagos""",5
2,3,"""Had a great time in #Quito but wish there were more options for vegetarians. @TravelEcuador https://t.co/quito_food""",3


<b> Interacción con la API</b>

In [6]:
import time

def call_LLM(prompt):
    # Cuerpo de la solicitud
    data = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt}
                ]
            }
        ]
    }
    
    # Encabezados de la solicitud
    headers = {
        "Content-Type": "application/json"
    }
    
    # Enviar la solicitud POST
    response = requests.post(url, headers=headers, data=json.dumps(data))
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Procesar la respuesta JSON
        result = response.json()
        result = json.dumps(result, indent=2)
        result = json.loads(result)
        time.sleep(5) 
        return result
    else:
        # Manejar errores
        print(f"Error: {response.status_code}")
        time.sleep(5) 
        return response.text

<b>Uso de la API</b>

In [7]:
result = call_LLM("Who is the Ecuador's President")
print(result)

{'candidates': [{'content': {'parts': [{'text': 'Guillermo Lasso'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}], 'usageMetadata': {'promptTokenCount': 8, 'candidatesTokenCount': 2, 'totalTokenCount': 10}, 'modelVersion': 'gemini-pro'}


## Uso de LLM para análisis de sentimientos

<b>Carga de datos de opiniones:</b>

In [8]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

opinions = pd.read_csv('../commentsEC.csv', names = ['id', 'text', 'rating'])
opinions.head(3)

,id,text,rating
0,1,"""Exploring the historic city of #Quito was amazing! Can't wait to come back! 🇪🇨 @VisitEcuador https://t.co/quito_adventures""",5
1,2,"""The Galápagos Islands are truly a paradise! 🐢😍 #Galapagos #WildlifeTravel https://t.co/galapagos""",5
2,3,"""Had a great time in #Quito but wish there were more options for vegetarians. @TravelEcuador https://t.co/quito_food""",3


<b>Definición del prompt para análisis de sentimientos a nivel de documento</b>

In [9]:

for opinion in opinions[:1].iterrows():
    prompt = """Given the following opinion { """ + opinion[1]['text'] + """}
            1) Make sentiment analysis and classify the sentence according the opinion expressed by the user.
            2) Provide an explanation used to make the inference.
            3) Return the output in csv format ("sentiment", "explanation")."""
    print(prompt, '\n', 40*'-','\n', )
    
    print(call_LLM(prompt))

Given the following opinion {  "Exploring the historic city of #Quito was amazing! Can't wait to come back! 🇪🇨 @VisitEcuador https://t.co/quito_adventures"}
            1) Make sentiment analysis and classify the sentence according the opinion expressed by the user.
            2) Provide an explanation used to make the inference.
            3) Return the output in csv format ("sentiment", "explanation"). 
 ---------------------------------------- 

{'candidates': [{'content': {'parts': [{'text': '| sentiment | explanation |\n|---|---|\n| positive | The user expresses a positive opinion about the experience of exploring the historic city of Quito and expresses a desire to return. |'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category'

<b>Definición del prompt para análisis de sentimientos a nivel de frase o aspecto</b>

In [56]:
##############################
##
##############################